In [1]:
# LIBRARIES #

import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk


output_notebook()

Loading BokehJS ...

In [2]:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers

m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/09 - etalonnage lot 10151801 19juil2019/films a 24h/'

m_nbOfFiles = 5
m_filesName = "scan"

m_dimViewer = 600

In [3]:
# READS THE IMAGE AND CONVERTS IT TO RGBA IMG FOR DISPLAY #

def readImgAndConvertToRGBA(filename):

    # reads the image using simpleITK:
    img = sitk.ReadImage(filename)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    array = sitk.GetArrayFromImage(img)
    
    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return array, rgba, sizex, sizey

In [4]:
# CONVERTS THE IMG TO RGBA FOR DISPLAY #

def convertToRGBA(array, sizex, sizey):

    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return rgba

In [5]:
# READS ALL IMAGES IN THE PATH AND RETURNS THE MEDIAN IMAGE #

def medianImage(path, filename, nbOfImgs):

    imgs = []
    
    # reads the images and store the matrix in imgs:
    sizex=0 
    sizey =0
    for i in range(nbOfImgs):
        im = sitk.ReadImage(path+filename+str(i+1)+'.tif')
        sizex = im.GetWidth()
        sizey = im.GetHeight()
        ar = sitk.GetArrayFromImage(im)
        imgs.append(ar)

    print('Image size:', sizex, 'x', sizey, 'x 3 layers (R,G,B)')
    print('')

    # creation and filling of median image:
    medianImg = np.empty((sizey,sizex,3))
    for i in range(sizex):
        for j in range(sizey):
            for k in range(3):
                a = []
                for l in range(nbOfImgs):
                    a.append(imgs[l][j][i][k])
                medianImg[j][i][k] = np.median(a)
    
    return medianImg, sizex, sizey

In [7]:
# READS THE IMAGE AND CALCULATES MEAN VALUES:

medianImg, sizex, sizey = medianImage(m_path, m_filesName, m_nbOfFiles)
rgbaimg = convertToRGBA(medianImg, sizex, sizey)
arrayimg = medianImg


#arrayimg, rgbaimg, sizex, sizey = readImgAndConvertToRGBA(m_path+m_filesName+'1'+'.tif')

p = figure(plot_width=int(m_dimViewer*0.95), plot_height=int(m_dimViewer*0.95*sizey/sizex), x_range=(0,sizex), y_range=(0,sizey), 
            title="Gafchromic image", toolbar_location="above")

p.image_rgba(image=[rgbaimg], x=[0], y=[0], dw=[sizex], dh=[sizey])

rectList = []
rectList.append([310,65,220,65])
rectList.append([310,162,220,65])
rectList.append([310,252,220,65])
rectList.append([310,337,220,65])
rectList.append([310,425,220,65])
rectList.append([310,510,220,65])
rectList.append([310,600,220,65])
rectList.append([310,690,220,65])

for i in range(len(rectList)):
    print('rectangle', str(i), ':') 
    p.rect(x=int(rectList[i][0]), y=rectList[i][1], width=rectList[i][2], height=rectList[i][3], fill_color="#0096ff", alpha=0.2)
    print('red value:', np.mean(arrayimg[int(rectList[i][1]-rectList[i][3]/2):int(rectList[i][1]+rectList[i][3]/2),
                           int(rectList[i][0]-rectList[i][2]/2):int(rectList[i][0]+rectList[i][2]/2), 0]))
    print('green value:', np.mean(arrayimg[int(rectList[i][1]-rectList[i][3]/2):int(rectList[i][1]+rectList[i][3]/2),
                           int(rectList[i][0]-rectList[i][2]/2):int(rectList[i][0]+rectList[i][2]/2), 1]))
    print('blue value:', np.mean(arrayimg[int(rectList[i][1]-rectList[i][3]/2):int(rectList[i][1]+rectList[i][3]/2),
                           int(rectList[i][0]-rectList[i][2]/2):int(rectList[i][0]+rectList[i][2]/2), 2]))
    print('')
    
show(p)

Image size: 593 x 738 x 3 layers (R,G,B)

rectangle 0 :
red value: 16092.862377622378
green value: 18257.656153846154
blue value: 19368.02013986014

rectangle 1 :
red value: 18312.667342657343
green value: 20850.317132867134
blue value: 21027.183076923076

rectangle 2 :
red value: 21730.672097902097
green value: 24670.43909090909
blue value: 23215.56027972028

rectangle 3 :
red value: 24253.952657342656
green value: 27324.22090909091
blue value: 24547.005244755244

rectangle 4 :
red value: 27846.07048951049
green value: 30874.42902097902
blue value: 26146.796083916084

rectangle 5 :
red value: 33692.313776223775
green value: 36055.80678321678
blue value: 28201.47244755245

rectangle 6 :
red value: 38397.25972027972
green value: 39655.56118881119
blue value: 29519.83685314685

rectangle 7 :
red value: 41821.00986013986
green value: 41984.984825174826
blue value: 30401.033846153845

